In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../src/")

In [2]:
from package.logger import Timed, rlog, setup
from package import storage

setup("INFO")

In [3]:
import os

import folium
import geopandas as gpd
import pandas as pd

from package import strtime
from package.osm import osm
from package.mcr.data import NetworkType, OSMData
from package.mcr5.mcr5 import MCR5
from package import storage

from package.geometa import GeoMeta
from package.mcr.data import NetworkType

In [4]:
city_id = "Berlin" # 'Koeln'
geo_meta_path = f"../data/geometa_{city_id.lower()}.pkl"
stops = f"../data/cleaned_{city_id.lower()}/stops.csv"
structs = f"../data/structs_{city_id.lower()}.pkl"
location_mapping_path = f"../data/location_mappings_{city_id.lower()}.pkl"
bicycle_base_path = f"../data/bicycle_locations_{city_id.lower()}/"
mcr5_output_path = f"../data/mcr5/{city_id.lower()}"
# bicycle_location_path = "../data/bicycle_locations/2022-11-01_09_01_00.csv"

In [5]:
geo_meta = GeoMeta.load(geo_meta_path)
# geo_data = OSMData(
# 	geo_meta,
# 	city_id,
# 	additional_network_types=[NetworkType.DRIVING, NetworkType.CYCLING],
# )
# nodes = geo_data.osm_nodes
# edges = geo_data.osm_edges
# driving_nodes, _, _ = geo_data.additional_networks[NetworkType.DRIVING]

In [6]:
lm_data = storage.read_any_dict(location_mapping_path)
location_mappings = lm_data["location_mappings"]
car_location_mappings = lm_data["car_location_mappings"]

In [7]:
from command.step_config import (
    get_bicycle_public_transport_config,
    get_car_only_config,
    get_bicycle_only_config,
    get_public_transport_only_config,
    get_walking_only_config,
)

In [8]:
configs = {}


def get_bicyle_public_transport_config_ready(bicycle_location_path, start_time):
    initial_steps, repeating_steps = get_bicycle_public_transport_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
        bicycle_price_function="next_bike_no_tariff",
        bicycle_location_path=bicycle_location_path,
        structs_path=structs,
        stops_path=stops,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
        "start_time": start_time,
    }


def get_car_only_config_ready():
    initial_steps, repeating_steps = get_car_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": car_location_mappings,
        "max_transfers": 1,
    }


def get_bicycle_only_config_ready(bicycle_location_path):
    initial_steps, repeating_steps = get_bicycle_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
        bicycle_price_function="next_bike_no_tariff",
        bicycle_location_path=bicycle_location_path,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
    }


def get_public_transport_only_config_ready(start_time):
    initial_steps, repeating_steps = get_public_transport_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
        structs_path=structs,
        stops_path=stops,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
        "start_time": start_time,
    }


def get_walking_only_config_ready():
    initial_steps, repeating_steps = get_walking_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 0,
    }

In [9]:
import itertools
import os

times = [
    "08:00:00",
    "12:00:00",
    "18:00:00",
]
bicycle_location_paths = [
    os.path.join(bicycle_base_path, path)
    for path in os.listdir(bicycle_base_path)
]

bicycle_public_transport_config_args = list(
    itertools.product(
        times,
        bicycle_location_paths,
    )
)

In [10]:
from functools import partial

for i, (time, bicycle_location_path) in enumerate(bicycle_public_transport_config_args):
    print(i, time, bicycle_location_path)
    configs[f"bicycle_public_transport_{i}"] = partial(
        get_bicyle_public_transport_config_ready, bicycle_location_path, time
    )

print()
for i, time in enumerate(times):
    print(i, time)
    configs[f"public_transport_{i}"] = partial(
        get_public_transport_only_config_ready, time
    )

print()
for i, bicycle_location_path in enumerate(bicycle_location_paths):
    print(i, bicycle_location_path)
    configs[f"bicycle_{i}"] = partial(
        get_bicycle_only_config_ready, bicycle_location_path
    )

configs["car"] = get_car_only_config_ready
configs["walking"] = get_walking_only_config_ready

0 08:00:00 ../data/bicycle_locations_berlin/2022-09-17_00_01_00.csv
1 08:00:00 ../data/bicycle_locations_berlin/2023-02-24_09_01_00.csv
2 08:00:00 ../data/bicycle_locations_berlin/2022-02-17_07_01_00.csv
3 08:00:00 ../data/bicycle_locations_berlin/2023-01-19_18_01_00.csv
4 08:00:00 ../data/bicycle_locations_berlin/2023-04-19_07_01_00.csv
5 08:00:00 ../data/bicycle_locations_berlin/2022-03-15_17_01_00.csv
6 08:00:00 ../data/bicycle_locations_berlin/2022-06-14_18_01_00.csv
7 08:00:00 ../data/bicycle_locations_berlin/2022-11-03_23_01_00.csv
8 12:00:00 ../data/bicycle_locations_berlin/2022-09-17_00_01_00.csv
9 12:00:00 ../data/bicycle_locations_berlin/2023-02-24_09_01_00.csv
10 12:00:00 ../data/bicycle_locations_berlin/2022-02-17_07_01_00.csv
11 12:00:00 ../data/bicycle_locations_berlin/2023-01-19_18_01_00.csv
12 12:00:00 ../data/bicycle_locations_berlin/2023-04-19_07_01_00.csv
13 12:00:00 ../data/bicycle_locations_berlin/2022-03-15_17_01_00.csv
14 12:00:00 ../data/bicycle_locations_berlin

In [11]:
if os.path.exists(mcr5_output_path):
    raise Exception("Output path already exists")

In [12]:
from datetime import datetime
import pickle

In [13]:
configs

{'bicycle_public_transport_0': functools.partial(<function get_bicyle_public_transport_config_ready at 0x7fd560d54040>, '../data/bicycle_locations_berlin/2022-09-17_00_01_00.csv', '08:00:00'),
 'bicycle_public_transport_1': functools.partial(<function get_bicyle_public_transport_config_ready at 0x7fd560d54040>, '../data/bicycle_locations_berlin/2023-02-24_09_01_00.csv', '08:00:00'),
 'bicycle_public_transport_2': functools.partial(<function get_bicyle_public_transport_config_ready at 0x7fd560d54040>, '../data/bicycle_locations_berlin/2022-02-17_07_01_00.csv', '08:00:00'),
 'bicycle_public_transport_3': functools.partial(<function get_bicyle_public_transport_config_ready at 0x7fd560d54040>, '../data/bicycle_locations_berlin/2023-01-19_18_01_00.csv', '08:00:00'),
 'bicycle_public_transport_4': functools.partial(<function get_bicyle_public_transport_config_ready at 0x7fd560d54040>, '../data/bicycle_locations_berlin/2023-04-19_07_01_00.csv', '08:00:00'),
 'bicycle_public_transport_5': func

In [14]:
runtimes = {}
for key, config in configs.items():
    start = datetime.now()
    rlog.info(f"Running MCR5 for {key}")

    config = config()
    mcr5 = MCR5(**config["init_kwargs"], max_processes=8)

    loaded_at = datetime.now()
    load_time = loaded_at - start

    output_path = os.path.join(mcr5_output_path, key)

    location_mappings = config["location_mappings"]

    start_time = config.get("start_time", "08:00:00")
    errors = mcr5.run(
        location_mappings,
        start_time=start_time,
        output_dir=output_path,
        max_transfers=config["max_transfers"],
    )

    run_time = datetime.now() - loaded_at
    total_time = datetime.now() - start
    runtimes[key] = {
        "load_time": load_time,
        "run_time": run_time,
        "total_time": total_time,
    }

with open(os.path.join(mcr5_output_path, "runtimes.pkl"), "wb") as f:
    pickle.dump(runtimes, f)

[14:20:50] INFO     Running MCR5 for bicycle_public_transport_0                                      ]8;id=404655;file:///tmp/ipykernel_3152471/542061263.py\542061263.py]8;;\:]8;id=284331;file:///tmp/ipykernel_3152471/542061263.py#4\4]8;;\

           INFO     Using existing OSM data                                                              ]8;id=388180;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=680750;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (walking)                                              ]8;id=90080;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=475232;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

[14:23:13] INFO     Using existing OSM data                                                              ]8;id=972293;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=188081;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (cycling)                                              ]8;id=489990;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=616096;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

[14:24:40] INFO     Fetching POI for runtime optimization                                         ]8;id=2894;file:///home/ppeter/repo/mcr-py/notebooks/../src/command/step_config.py\step_config.py]8;;\:]8;id=158661;file:///home/ppeter/repo/mcr-py/notebooks/../src/command/step_config.py#51\51]8;;\

           INFO     Fetching POI for runtime optimization done (0.21 seconds)                     ]8;id=840469;file:///home/ppeter/repo/mcr-py/notebooks/../src/command/step_config.py\step_config.py]8;;\:]8;id=759025;file:///home/ppeter/repo/mcr-py/notebooks/../src/command/step_config.py#51\51]8;;\

[14:26:56] INFO     Reading stops                                                           ]8;id=568820;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py\public_transport.py]8;;\:]8;id=266586;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py#144\144]8;;\

[14:26:57] INFO     Reading stops done (1.18 seconds)                                       ]8;id=113473;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py\public_transport.py]8;;\:]8;id=162047;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py#144\144]8;;\

ValueError: invalid literal for int() with base 10: 'de:12054:900230999:1:50'

In [ ]:
print("Ready")

In [ ]:
import pickle
import os

# errors = storage.read_any_dict("../data/mcr5/Koeln/bicycle_public_transport/errors.pkl")
# errors